In [ ]:
from PIL import Image
import numpy as np
import os
import cv2

def auto_canny(image, sigma=0.33):
    """
    Automatic Canny edge detection.
    
    Args:
        image (numpy array): The image in which to detect edges.
        sigma (float): The parameter to adjust sensitivity of edge detection.
    
    Returns:
        numpy array: The edged image where edges are highlighted.
    """
    # Compute the median of the single channel pixel intensities
    v = np.median(image)
    
    # Apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    
    return edged

def crop_and_square_image(image_path, output_path):
    """
    Crop the image to the main content and make it square.
    
    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the cropped and squared image.
    """
    # Open and convert the image to grayscale
    with Image.open(image_path) as img:
        img_gray = img.convert('L')
        img_np = np.array(img_gray)

        # Detect edges in the image
        edges = auto_canny(img_np)
        
        # Find points where the image has edges
        if np.any(edges):
            points = np.argwhere(edges)
            top_left = points.min(axis=0)
            bottom_right = points.max(axis=0)
            bbox = (top_left[1], top_left[0], bottom_right[1], bottom_right[0])

            # Crop the image to the bounding box that contains the edges
            img_cropped = img.crop(bbox)

            # Determine the size to make a square image
            width = bbox[2] - bbox[0]
            height = bbox[3] - bbox[1]
            side_length = max(width, height)
            new_image = Image.new('L', (side_length, side_length), 255)
            
            # Center the cropped image in the new square canvas
            new_image.paste(img_cropped, ((side_length - width) // 2, (side_length - height) // 2))

            # Save the newly created square image
            new_image.save(output_path)

def batch_crop_images(input_folder, output_folder):
    """
    Process multiple images to crop and square them.
    
    Args:
        input_folder (str): Directory containing the images to process.
        output_folder (str): Directory where processed images will be saved.
    """
    # Iterate through each file in the input directory
    for img_name in os.listdir(input_folder):
        if img_name.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, img_name)
            output_path = os.path.join(output_folder, img_name)
            
            # Process each image to crop and square it
            crop_and_square_image(img_path, output_path)
            print(f"Cropped and squared {img_name}")

# Example usage
input_folder = "./new_test - Copy"
output_folder = './new_ouyang_training11'
batch_crop_images(input_folder, output_folder)
